In [1]:
!pip install openpyxl

In [4]:
%matplotlib inline
from catboost import CatBoostClassifier
from itertools import combinations
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np 
import optuna
import pandas as pd 
import re
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report, plot_confusion_matrix, f1_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

In [322]:
pd.__version__

In [5]:
VENDORS_DICT_CUSTOM_PATH = "/kaggle/input/hacksai-3/ved_dict.csv"
DATASET_PATH = "/kaggle/input/hacksai-3/dataset.xlsx"

# First look

In [6]:
ved_dict = pd.read_csv(VENDORS_DICT_CUSTOM_PATH, sep=";") \
    .dropna(subset=["VED"])

for code_col in ["GRUPPA", "TOV_POZ", "SUB_POZ", "VED", "RAZDEL"]:
    ved_dict.loc[ved_dict[code_col].notna(), code_col] =\
        ved_dict.loc[ved_dict[code_col].notna(), code_col].astype(int).astype(str)


ved_dict.head(5)

In [60]:
whole_df = pd.read_excel(DATASET_PATH)

whole_df.describe()

In [61]:
whole_df.info()

In [36]:
whole_df.head(5)

In [62]:
whole_df["Номер продукции"] = whole_df["Номер продукции"].str.replace(" Продукция", "")

df = whole_df.copy()

In [63]:
df = df[~df[df.columns[1:]].duplicated()].reset_index(drop=True)

In [64]:
df = df.dropna(subset=["Коды ТН ВЭД ЕАЭС"]).reset_index(drop=True)

In [65]:
df["Коды ТН ВЭД ЕАЭС"] = df["Коды ТН ВЭД ЕАЭС"].astype(str) \
    .str.split("; ") \
    .apply(set) \
    .apply(list)

df["Коды ТН ВЭД ЕАЭС"].apply(len).hist(bins=20, color="grey")
plt.show()

In [66]:
TN_VED_THRESHOLD = df["Коды ТН ВЭД ЕАЭС"].apply(len).quantile(.95)
print(TN_VED_THRESHOLD)

In [72]:
df = df[~(df["Коды ТН ВЭД ЕАЭС"].apply(len) > TN_VED_THRESHOLD)].reset_index(drop=True)

In [73]:
df = df.dropna(subset=["Технические регламенты"]).reset_index(drop=True)

In [74]:
df[~df["Технические регламенты"].str.contains(r"[\w]{2} [\w]{2,4} [\d]{1,4}/[\d]{4}", regex=True)]

In [75]:
df["Технические регламенты"] = df["Технические регламенты"].str.split("; ") \
    .apply(lambda x: list(set([i.strip() for i in x])))

df["Технические регламенты"].apply(len).hist(bins=20, color="grey")
plt.show()

In [76]:
df = df.dropna(subset=["Группа продукции"]).reset_index(drop=True)

In [77]:
df["Группа продукции"] = df["Группа продукции"].str.split(";") \
    .apply(lambda x: list(set([i.strip() for i in x])))

df["Группа продукции"].apply(len).hist(bins=20, color="grey")
plt.show()

In [78]:
PRODUCT_GROUP_THRESHOLD = df["Группа продукции"].apply(len).quantile(.95)
print(PRODUCT_GROUP_THRESHOLD)

In [79]:
df = df[~(df["Группа продукции"].apply(len) > PRODUCT_GROUP_THRESHOLD)].reset_index(drop=True)

In [80]:
# explode rows with multiple codes or groups and etc.

for col in df.columns[1:]:
    df = df.explode(col)
    
df = df.reset_index(drop=True)
df.shape

In [81]:
df = df.merge(
        ved_dict[["RAZDEL", "GRUPPA", "TOV_POZ", "VED"]].drop_duplicates() \
            .fillna("-1") \
            .rename(columns={"VED": "Коды ТН ВЭД ЕАЭС"}), 
        on="Коды ТН ВЭД ЕАЭС", how="left"
    ).fillna("-1")

df.sample(3)

# Anomaly detection


## 1. Find samples with anomaly using SVD and DBSCAN

In [83]:
data = df[df.columns[:4].tolist() + df.columns[-3:].tolist()].copy()

data.head(5)

In [84]:
data[data.columns[2:]].describe()

In [85]:
%%time
ohe = OneHotEncoder()

ohe.fit(data.drop(["Коды ТН ВЭД ЕАЭС", "Номер продукции"], axis=1))

X_cat = ohe.transform(data[ohe.feature_names_in_])
X_cat.shape

In [86]:
%%time
svd = TruncatedSVD(
    n_components=3, 
    n_iter=20, 
    random_state=42
)

svd.fit(X_cat)

X_cat_mtx = svd.transform(X_cat)
X_cat_mtx.shape

In [87]:
cmap = mp.cm.get_cmap("Set1")
for eps in tqdm([5e-2, 5e-3, 5e-4, 1e-4]):
    for min_s in [5, 10, 15]:
        clusterizer = DBSCAN(
            eps=eps, 
            min_samples=min_s
        )

        clusterizer.fit(X_cat_mtx)

        clusters = clusterizer.labels_
        
        print(f"Epsilon: {eps}, min samples: {min_s}.")
        plt.figure(figsize=(10, 10))
        plt.title(
            "Detection results: %.0f anomaly over %.0f samples" % (sum(clusters == -1), clusters.shape[0]), 
            fontsize=15
        )
        plt.scatter(
            x=X_cat_mtx[:, 0], y=X_cat_mtx[:, 1], 
            cmap=cmap, s=.2,
            c=(clusters != -1).astype(int)
        )
        plt.show()

In [94]:
%%time
clusterizer = DBSCAN(
    eps=5e-4, 
    min_samples=10
)

clusterizer.fit(X_cat_mtx)

clusters = clusterizer.labels_

In [95]:
data["outlier"] = (clusters == -1).astype(int)
data["outlier"].value_counts()

In [96]:
(sum(clusters == -1) / data.shape[0]) * 100

## 2. Co-occurences

In [97]:
def pmi(dataframe, x, y):
    d = dataframe.copy()
    d['f_x'] = d.groupby(x)[x].transform('count')
    d['f_y'] = d.groupby(y)[y].transform('count')
    d['f_xy'] = d.groupby([x, y])[x].transform('count')
    d[f'pmi_{x}/{y}'] = np.log(len(d.index) * d['f_xy'] / (d['f_x'] * d['f_y']) )
    return d.drop(["f_x", "f_y", "f_xy"], axis=1)

features = data.columns[-6:-1]

for f_1, f_2 in combinations(features, 2):
    if f_1 != f_2:
        data = pmi(data, f_1, f_2)

In [98]:
((data[data.columns[-10:]] < 0).sum(axis=1) == 0).mean()

# it seems like it doesn't work (can't filter anomaly using pmi)

## 3. Train classifier for automatically anomaly detection

In [99]:
data.info()

In [100]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(["Номер продукции", "Коды ТН ВЭД ЕАЭС", "outlier"], axis=1), data["outlier"],
    random_state=42, test_size=.2, stratify=data["outlier"]
)
X_train.shape, X_test.shape

In [101]:
def objective(trial):
    param = {
        "objective"        : trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth"            : trial.suggest_int("depth", 3, 6)
    }

    gbm = CatBoostClassifier(**param)

    gbm.fit(
        X_train, y_train, 
        eval_set=[(X_test, y_test)], verbose=0, 
        early_stopping_rounds=50, cat_features=[0, 1, 2, 3, 4]
    )

    preds = gbm.predict(X_test)
    f1_macro = f1_score(y_test, preds, average="macro")
    return f1_macro

In [102]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15, n_jobs=5)

In [103]:
print("Best parameters after tuning")
print(study.best_params)

In [104]:
clf = CatBoostClassifier(**study.best_params)
clf.fit(
    X_train, y_train, eval_set=[(X_test, y_test)], verbose=1, early_stopping_rounds=50,
    cat_features=[0, 1, 2, 3, 4]
)

In [105]:
y_pred = clf.predict(X_test)
y_probs = clf.predict_proba(X_test)

print(classification_report(y_test, y_pred))

In [106]:
plot_confusion_matrix(clf, X_test, y_test, cmap="Reds")
plt.show()

In [107]:
plt.title("Degree of model uncertainty in predictions", fontsize=15)
plt.hist(np.abs(y_probs[:, 0] - y_probs[:, 1]), bins=20, color="grey")
plt.xlabel("ABS difference in certainty between POS and NEG classes")
plt.ylabel("Number of samples")
plt.show()

In [108]:
left_q = np.quantile(np.abs(y_probs[:, 0] - y_probs[:, 1]), .05)
print(left_q)
print((np.abs(y_probs[:, 0] - y_probs[:, 1]) < left_q).mean())

In [109]:
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1])
gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
BEST_THRESHOLD = thresholds[ix]

print("Best Threshold=%f, G-Mean=%.3f" % (BEST_THRESHOLD, gmeans[ix]))
plt.plot([0,1], [0,1], linestyle="--", label="No Skill", color="red")
plt.plot(fpr, tpr, marker=".", label="Catboost", color="grey", markersize=3)
plt.scatter(fpr[ix], tpr[ix], marker="o", color="black", label="Best")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [110]:
print(classification_report(y_test, y_probs[:, 1] > BEST_THRESHOLD))

# works worse with threshold

In [111]:
X_test.loc[y_test != y_pred, X_test.columns[:5]].describe()

In [112]:
X_test.loc[y_test == y_pred, X_test.columns[:5]].describe()

In [113]:
X_test[X_test.columns[:5]].describe()

In [114]:
X_test.loc[y_test != y_pred, X_test.columns[-5:]].describe()

In [115]:
X_test.loc[y_test == y_pred, X_test.columns[-5:]].describe()

In [116]:
import shap
shap.initjs()

In [117]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_train)

In [255]:
neg_samples = ["14283", "38206", "2483", "49128", "19872", "33395", "43302", "64399", "35771", "15399"]
neg_samples_df = whole_df.loc[
    whole_df["Номер продукции"].isin(neg_samples),
    ["Номер продукции", "Коды ТН ВЭД ЕАЭС", "Технические регламенты", "Группа продукции", "Общее наименование продукции"]
].reset_index(drop=True)

neg_idxs = [n[0][0]
    for n in [np.where(X_train.index == i) 
     for i in data[data["Номер продукции"].isin(neg_samples)].index.tolist()]
 if len(n[0]) > 0
]

In [256]:
idxs_dict = data.loc[data["Номер продукции"].isin(neg_samples), "Номер продукции"].to_dict()
err_dict = {}

for neg_idx in neg_idxs:
    l = np.array(clf.feature_names_)[np.argsort(shap_values[neg_idx, :])[::-1][:3]].tolist()
    prod_num = idxs_dict.get(X_train.iloc[[neg_idx]].index[0])
    err_dict[prod_num] = list(set(err_dict.get(prod_num, []) + l))
    
neg_samples_df["Причина ошибок"] = neg_samples_df["Номер продукции"].map(err_dict.get)
neg_samples_df.to_csv("neg_sample_demo.csv", index=False, encoding="utf-8", sep=";")

In [258]:
shap.force_plot(
    explainer.expected_value, shap_values[0, :], X_train.iloc[0, :], 
    text_rotation=45, figsize=(5, 3)
)

In [259]:
shap.force_plot(
    explainer.expected_value, shap_values[40080, :], X_train.iloc[40080, :], 
    text_rotation=45, figsize=(5,3)
)

In [262]:
shap.summary_plot(shap_values, X_train)
plt.show()

In [263]:
shap.force_plot(
    explainer.expected_value, 
    shap_values[np.random.choice(shap_values.shape[0], 50, replace=False), :], 
    X_train.iloc[np.random.choice(shap_values.shape[0], 50, replace=False), :]
)

In [264]:
final_clf = CatBoostClassifier(**study.best_params)
final_clf.fit(
    data.drop(["Номер продукции", "Коды ТН ВЭД ЕАЭС", "outlier"], axis=1), 
    data["outlier"], cat_features=[0, 1, 2, 3, 4]
)
final_clf.save_model(
    "anomaly_detector.model",
    format="cbm",
    export_parameters=None,
    pool=None
)

In [265]:
data.head(5)

In [266]:
pmi_features = data[features.tolist() + data.columns[-10:].tolist()].drop_duplicates() \
    .reset_index(drop=True)
pmi_features.to_csv("pmi_features.csv", encoding="utf-8", sep=";", index=False)

In [303]:
from catboost import CatBoostClassifier
import pandas as pd


def load_model(path):
    """ Load classifier
    
    Args:
        path - path to model.
    """
    clf = CatBoostClassifier()
    clf.load_model(path)
    return clf


def load_pmi(path):
    """ Load pmi historical stats
    
    Args:
        path - path stats.
    Returns:
        (pd.DataFrame)
    """
    return pd.read_csv(path, sep=";")


def load_ved_thesaurus(path):
    """ Load ved thesaurus
    
    Args:
        path - path to ved thesaurus.
    Returns:
        (pd.DataFrame)
    """
    return pd.read_csv(path, sep=";")

    
def pmi_predict(dataframe, x, y):
    """Calculate PMI for income data based on historical stats.
    
    Args:
        dataframe_hist - historical PMI stats;
        dataframe - new data to calculation;
        x - column left;
        y - column right.
    Returns:
        (pd.DataFrame)
    """
    h_d = pmi_features[[x, y, f"pmi_{x}/{y}"]] \
        .drop_duplicates() \
        .reset_index(drop=True)
    dataframe = dataframe.merge(
        h_d, on=[x, y], how="left"
    ).fillna({f"pmi_{x}/{y}": -5})
    return dataframe


def add_ved_info(dataframe):
    """ Add VED main categories.
    
    Args:
        dataframe - income data.
    Returns:
        (pd.DataFrame)
    """
    return dataframe.merge(
        ved_dict[["VED", "RAZDEL", "GRUPPA", "TOV_POZ"]].drop_duplicates() \
            .rename(columns={"VED": "Коды ТН ВЭД ЕАЭС"}) \
            .astype(str),
        on=["Коды ТН ВЭД ЕАЭС"], how="left"
    ).fillna("-1")


def predict(dataframe):
    """ Detect outliers in dataframe attributes.
    
    Args:
        dataframe - income data.
    Returns:
        (pd.DataFrame)
    """
    d = dataframe.copy()
    if dataframe[["Технические регламенты", "Группа продукции"]].isna().sum().sum() > 0:
        return 1
    
    idxs = ["Номер продукции"]
    features = ["Технические регламенты", "Группа продукции", "RAZDEL", "GRUPPA", "TOV_POZ"]
    
    dataframe = add_ved_info(dataframe)
    dataframe = dataframe[idxs + features]
    
    for f_1, f_2 in combinations(features, 2):
        if f_1 != f_2:
            dataframe = pmi_predict(dataframe, f_1, f_2)
            
    dataframe = dataframe.astype({"Технические регламенты": str}) 
    dataframe["Технические регламенты"] = dataframe["Технические регламенты"].str.split("; ")
    dataframe["Группа продукции"] = dataframe["Группа продукции"].str.split("; ")
    dataframe = dataframe.explode("Технические регламенты") \
        .dropna() \
        .explode("Группа продукции") \
        .dropna() \
        .reset_index(drop=True)
    
    dataframe["outlier"] = final_clf.predict(dataframe[final_clf.feature_names_])
    
    return d.merge(dataframe.groupby(idxs)["outlier"].max().reset_index(), on=idxs)

In [304]:
s = whole_df.iloc[[0]]
s

In [305]:
%%time
predict(s)

In [307]:
whole_preds = final_clf.predict(data.drop(["Номер продукции", "Коды ТН ВЭД ЕАЭС", "outlier"], axis=1))
data["outlier"] = whole_preds

whole_preds = data[["Номер продукции", "outlier"]] \
    .groupby("Номер продукции")["outlier"].max().reset_index()

whole_df = whole_df.merge(
    whole_preds,
    on=["Номер продукции"], how="left"
).fillna({"outlier": 1}).astype({"outlier": int})

whole_df.to_csv("outlier_whole_preds.csv", index=False, sep=";")